# HW3 Approaches in anomaly detection

## Reading the input data

In [1]:
from sklearn.neighbors import KernelDensity
from sklearn.cross_validation import train_test_split
import numpy as np
import scipy.io
mat = scipy.io.loadmat('oc_514.mat')
train = mat['x']
xtrain = train[0,0][0]

# Commented split of train and test as using k-fold cross validation approach for outlier detection.
X_train, X_test = train_test_split(
   xtrain, test_size=0.33, random_state=42)


## Anomaly Detection using KDE

In [2]:

#Using Gaussian Kernel Density Estimation
def GaussianKDE(X_train, X_test):
    #pdf = KernelDensity(bandwidth=0.25, kernel='linear')
    pdf = KernelDensity(bandwidth=0.1, kernel='gaussian')
    pdf.fit(X_train)
    resTrain = []
    resTest = []
    
    for data in X_train[:,:]:
        resTrain.append(np.exp(pdf.score_samples(data))[0])
    
    for data in X_test[:,:]:
        resTest.append(np.exp(pdf.score_samples(data))[0])
    
    nresTrain = np.array(resTrain)
    nresTest = np.array(resTest)
    
    # Assumption: "Normal data instances occur in high probability
    # regions of a stochastic model, while anomalies occur in the low
    # probability regions of the stochastic model."
    return nresTrain[nresTrain <= 0.05].size, nresTest[nresTest <= 0.05].size

In [3]:

# Cross Validation using K-Fold so that every data sample contributes in Training

from sklearn.cross_validation import KFold

X_train = xtrain
#print X_train.shape
n_folds = 5
kf = KFold(420, n_folds)
error_train = error_test = 0
for train, test in kf:
    #print train,test
    n_error_train, n_error_test = GaussianKDE(X_train[train,:], X_train[test,:])
    error_test += n_error_test
    error_train += n_error_train
    
print float(error_train)/n_folds , float(error_test)/n_folds


0.0 81.2


In [4]:
# Split the train and test set in the ratio of 3:1

n_error_train, n_error_test = GaussianKDE(X_train[:315,:], X_train[315:,:])


print n_error_train, n_error_test

0 105


### Performance of KDE for Anomaly Detection

In [5]:
print "Using KDE we got", (error_train + error_test)/n_folds , "anomalies out of 420 samples. This performs poorly as the number of anomalies is higher than the expected number of 183 as defined in the webpage of the dataset. Kernel Density Estimation (KDE) can be performed in any number of dimensions, though in practice the curse of dimensionality causes its performance to degrade in high dimensions which is the case with our dataset as it has 278 features." 

Using KDE we got 81 anomalies out of 420 samples. This performs poorly as the number of anomalies is higher than the expected number of 183 as defined in the webpage of the dataset. Kernel Density Estimation (KDE) can be performed in any number of dimensions, though in practice the curse of dimensionality causes its performance to degrade in high dimensions which is the case with our dataset as it has 278 features.


## Anomaly Detection using One Class SVM

In [6]:
#xtrain = train[0,0][0]
def OutlierUsingOneClassSVM(X_train, X_test):

    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.font_manager
    from sklearn import svm
    import scipy.io as sio

    #xx, yy = np.meshgrid(np.linspace(-5, 5, 500), np.linspace(-5, 5, 500))

    # Generate train data
    #X = 0.3 * np.random.randn(100, 2)
    #X_train = np.r_[X + 2, X - 2]
    # Generate some regular novel observations
    #X = 0.3 * np.random.randn(20, 2)
    #X_test = np.r_[X + 2, X - 2]
    # Generate some abnormal novel observations
    #X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))


    # fit the model
    #clf = svm.OneClassSVM(nu=0.2, kernel="rbf", gamma=0.1)
    clf = svm.OneClassSVM(nu=0.435, kernel="linear", gamma=0.1)
    clf.fit(X_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    #y_pred_outliers = clf.predict(X_outliers)
    n_error_train = y_pred_train[y_pred_train == -1].size
    n_error_test = y_pred_test[y_pred_test == -1].size
    #n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

    # Plots
    # plot the line, the points, and the nearest vectors to the plane
#     Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
#     Z = Z.reshape(xx.shape)

#     plt.title("Novelty Detection")
#     plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.Blues_r)
#     a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='red')
#     plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='orange')

#     b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white')
#     b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='green')
#     # #c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='red')
#     plt.axis('tight')
#     plt.xlim((-5, 5))
#     plt.ylim((-5, 5))
#     plt.legend([a.collections[0], b1, b2], #, c],
#                ["learned frontier", "training observations",
#                 "new regular observations", "new abnormal observations"],
#                loc="upper left",
#                prop=matplotlib.font_manager.FontProperties(size=11))
#     plt.xlabel(
#         "error train: %d/200 ; errors novel regular: %d/40 ; "
#         "errors novel abnormal:"
#         % (n_error_train, n_error_test)) #, n_error_outliers))
#     plt.show()

    return n_error_train, n_error_test


In [7]:
# Cross Validation using K-Fold so that every data sample contributes in Training

from sklearn.cross_validation import KFold

X_train = xtrain
print X_train.shape
n_folds = 5
kf = KFold(420, n_folds)
error_train = error_test = 0
for train, test in kf:
    #print train,test
    n_error_train, n_error_test = OutlierUsingOneClassSVM(X_train[train,:], X_train[test,:])
    #print error_train, error_test
    #print n_error_train, n_error_test
    error_train += n_error_train
    error_test += n_error_test
    
print float(error_train)/n_folds, float(error_test)/n_folds


(420L, 278L)
146.8 37.0


In [8]:
# Split the train and test set in the ratio of 4:1
n_error_train, n_error_test = OutlierUsingOneClassSVM(X_train[:336,:], X_train[336:,:])

print n_error_train, n_error_test


147 37


### Performance of One Class SVM for Anomaly Detection

In [9]:
print "Using One Class SVM we got", (n_error_train + n_error_test) , "anomalies out of 420 samples. This performs well after tuning the kernel to be linear and with appropriate parameters as the number of anomalies matches the expected number of 183 as defined in the webpage of the dataset." 


Using One Class SVM we got 184 anomalies out of 420 samples. This performs well after tuning the kernel to be linear and with appropriate parameters as the number of anomalies matches the expected number of 183 as defined in the webpage of the dataset.


# Local Outlier Factor (LOF)

In [10]:

from __future__ import division

# def euclideanDistance(point1, point2):
#     # Computes the Euclidean distance between two objects or points. 
#     # compute RMSE (root mean squared error)
#     difference = point1 - point2
#     rmse = (sum(map(lambda x: x**2, difference)) / len(difference))**0.5
#     return rmse

def euclideanDistance(instance1, instance2):
    """Computes the distance between two instances. Instances should be tuples of equal length.
    Returns: Euclidean distance
    Signature: ((attr_1_1, attr_1_2, ...), (attr_2_1, attr_2_2, ...)) -> float"""
    def detect_value_type(attribute):
        """Detects the value type (number or non-number).
        Returns: (value type, value casted as detected type)
        Signature: value -> (str or float type, str or float value)"""
        from numbers import Number
        attribute_type = None
        if isinstance(attribute, Number):
            attribute_type = float
            attribute = float(attribute)
        else:
            attribute_type = str
            attribute = str(attribute)
        return attribute_type, attribute
    # check if instances are of same length
    if len(instance1) != len(instance2):
        raise AttributeError("Instances have different number of arguments.")
    # init differences vector
    differences = [0] * len(instance1)
    # compute difference for each attribute and store it to differences vector
    for i, (attr1, attr2) in enumerate(zip(instance1, instance2)):
        type1, attr1 = detect_value_type(attr1)
        type2, attr2 = detect_value_type(attr2)
        # raise error is attributes are not of same data type.
        if type1 != type2:
            raise AttributeError("Instances have different data types.")
        if type1 is float:
            # compute difference for float
            differences[i] = attr1 - attr2
        else:
            # compute difference for string
            if attr1 == attr2:
                differences[i] = 0
            else:
                differences[i] = 1
    # compute RMSE (root mean squared error)
    rmse = (sum(map(lambda x: x**2, differences)) / len(differences))**0.5
    return rmse

class LOF:
    """Helper class for performing LOF computations and instances normalization."""
    def __init__(self, instances, normalize=True, distance_function=euclideanDistance):
        self.instances = instances
        self.normalize = normalize
        self.distance_function = distance_function
        if normalize:
            self.normalize_instances()

    def compute_instance_attribute_bounds(self):
        min_values = [float("inf")] * len(self.instances[0]) #n.ones(len(self.instances[0])) * n.inf 
        max_values = [float("-inf")] * len(self.instances[0]) #n.ones(len(self.instances[0])) * -1 * n.inf
        for instance in self.instances:
            min_values = tuple(map(lambda x,y: min(x,y), min_values,instance)) #n.minimum(min_values, instance)
            max_values = tuple(map(lambda x,y: max(x,y), max_values,instance)) #n.maximum(max_values, instance)
        self.max_attribute_values = max_values
        self.min_attribute_values = min_values
            
    def normalize_instances(self):
        """Normalizes the instances and stores the infromation for rescaling new instances."""
        if not hasattr(self, "max_attribute_values"):
            self.compute_instance_attribute_bounds()
        new_instances = []
        for instance in self.instances:
            new_instances.append(self.normalize_instance(instance)) # (instance - min_values) / (max_values - min_values)
        self.instances = new_instances
        
    def normalize_instance(self, instance):
        return tuple(map(lambda value,max,min: (value-min)/(max-min) if max-min > 0 else 0, 
                         instance, self.max_attribute_values, self.min_attribute_values))
        
    def local_outlier_factor(self, min_pts, instance):
        """The (local) outlier factor of instance captures the degree to which we call instance an outlier.
        min_pts is a parameter that is specifying a minimum number of instances to consider for computing LOF value.
        Returns: local outlier factor
        Signature: (int, (attr1, attr2, ...), ((attr_1_1, ...),(attr_2_1, ...), ...)) -> float"""
        if self.normalize:
            instance = self.normalize_instance(instance)
        return local_outlier_factor(min_pts, instance, self.instances, distance_function=self.distance_function)

def k_distance(k, instance, instances, distance_function=euclideanDistance):
    #TODO: implement caching
    """Computes the k-distance of instance as defined in paper. It also gatheres the set of k-distance neighbours.
    Returns: (k-distance, k-distance neighbours)
    Signature: (int, (attr1, attr2, ...), ((attr_1_1, ...),(attr_2_1, ...), ...)) -> (float, ((attr_j_1, ...),(attr_k_1, ...), ...))"""
    distances = {}
    for instance2 in instances:
        distance_value = distance_function(instance, instance2)
        if distance_value in distances:
            distances[distance_value].append(instance2)
        else:
            distances[distance_value] = [instance2]
    distances = sorted(distances.items())
    neighbours = []
    [neighbours.extend(n[1]) for n in distances[:k]]
    return distances[k - 1][0], neighbours

def reachability_distance(k, instance1, instance2, instances, distance_function=euclideanDistance):
    """The reachability distance of instance1 with respect to instance2.
    Returns: reachability distance
    Signature: (int, (attr_1_1, ...),(attr_2_1, ...)) -> float"""
    (k_distance_value, neighbours) = k_distance(k, instance2, instances, distance_function=distance_function)
    return max([k_distance_value, distance_function(instance1, instance2)])

def local_reachability_density(min_pts, instance, instances, **kwargs):
    """Local reachability density of instance is the inverse of the average reachability 
    distance based on the min_pts-nearest neighbors of instance.
    Returns: local reachability density
    Signature: (int, (attr1, attr2, ...), ((attr_1_1, ...),(attr_2_1, ...), ...)) -> float"""
    (k_distance_value, neighbours) = k_distance(min_pts, instance, instances, **kwargs)
    reachability_distances_array = [0]*len(neighbours) #n.zeros(len(neighbours))
    for i, neighbour in enumerate(neighbours):
        reachability_distances_array[i] = reachability_distance(min_pts, instance, neighbour, instances, **kwargs) 
    return len(neighbours) / sum(reachability_distances_array)

def local_outlier_factor(min_pts, instance, instances, **kwargs):
    """The (local) outlier factor of instance captures the degree to which we call instance an outlier.
    min_pts is a parameter that is specifying a minimum number of instances to consider for computing LOF value.
    Returns: local outlier factor
    Signature: (int, (attr1, attr2, ...), ((attr_1_1, ...),(attr_2_1, ...), ...)) -> float"""
    (k_distance_value, neighbours) = k_distance(min_pts, instance, instances, **kwargs)
    instance_lrd = local_reachability_density(min_pts, instance, instances, **kwargs)
    lrd_ratios_array = [0]* len(neighbours)
    for i, neighbour in enumerate(neighbours):
        instances_without_instance = set(instances)
        instances_without_instance.discard(neighbour)
        neighbour_lrd = local_reachability_density(min_pts, neighbour, instances_without_instance, **kwargs)
        lrd_ratios_array[i] = neighbour_lrd / instance_lrd
    return sum(lrd_ratios_array) / len(neighbours)

def outliers(k, instances, **kwargs):
    """Simple procedure to identify outliers in the dataset."""
    instances_value_backup = instances
    outliers = []
    for i, instance in enumerate(instances_value_backup):
        instances = list(instances_value_backup)
        instances.remove(instance)
        l = LOF(instances, **kwargs)
        value = l.local_outlier_factor(k, instance)
        if value > 1:
            outliers.append({"lof": value, "instance": instance, "index": i})
    outliers.sort(key=lambda o: o["lof"], reverse=True)
    return outliers


In [ ]:
train = X_train[:336,:]
lof = LOF(train)

for instance in X_train[336:,:]:
    value = lof.local_outlier_factor(10, instance)
    print value, instance
